In [1]:
%load_ext autoreload
%autoreload 2

## Save table with results

In [2]:
experiment_ids = 'f1'
timepoints = 'all'
regions = 'cusanovich_dm6_peaks_1kb'
correction = 'wasp'

In [3]:
label = '_'.join([experiment_ids, timepoints, 'windows'])
wasp_corrected = True if correction == 'wasp' else False

## Imports

In [4]:
# general
import sys
import os

In [5]:
import re

In [6]:
# tools
import numpy as np
import pandas as pd
import scanpy as sc
import scipy
from scipy.stats import probplot

from dali.utils.stats import apply_fdr_bh

sc.settings.verbosity = 3
pd.set_option("display.max_columns", None)

In [7]:
# plotting
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib_venn as venn

%config InlineBackend.figure_format = 'retina'

In [8]:
# local
sys.path.append('..')
from utils import settings
from utils import dataloader

## Global variables

In [9]:
DALIHOM_PVAL = 'DALIHOM'
DALIHET_PVAL = 'DALIHET_VAE'
DALIHET_LINEAGE_PVAL = lambda x : 'DALIHET_TIME_%s' % x
DALIJOINT_PVAL = 'DALIJOINT'
DALIJOINT_RHO = 'DALIJOINT_RHO'

## Load processed anndata

In [10]:
adata_total = sc.read(os.path.join(settings.DATA_DIR, label, 'total_counts_vae_processed.h5ad'))

Load anndata with GP estimates

In [11]:
df_all = pd.DataFrame()
adatas_allelic = dict()

for exp_id in settings.F1_EXP_IDS:
    # all tested peaks
    fname = '_'.join([exp_id, regions, correction, 'allelic_counts.h5ad'])
    adata = sc.read(os.path.join(settings.DATA_DIR, label, fname))
    adata_var = adata.var
    adata_var['exp_id'] = exp_id
    adata_var['base_rate'] = (adata.X.A.sum(0) / adata.layers['allelic_total'].A.sum(0)).ravel()
    
    # with GP estimates
    fname = '_'.join([exp_id, regions, correction, 'allelic_counts_processed.h5ad'])
    adata = sc.read(os.path.join(settings.DATA_DIR, label, fname))
    adata_var = pd.merge(adata_var, adata.var['qdiff_10'], left_index=True, right_index=True, how='outer', sort=False)
    
    adata_var = adata_var.reset_index()
    df_all = pd.concat([df_all, adata_var], ignore_index=True)
    adatas_allelic[exp_id] = adata

In [12]:
df_all['TSS_1kb_peak'] = df_all['TSS_1kb_peak'].replace({1: 'proximal', 0: 'distal'}).astype('category')

In [13]:
df_all['Het. AI'] = df_all[DALIHET_PVAL + '_bh'] < .1
df_all['Hom. AI'] = df_all[DALIHOM_PVAL + '_bh'] < .1
df_all['Joint. AI'] = df_all[DALIJOINT_PVAL + '_bh'] < .1

In [14]:
pval_cols = [DALIHET_PVAL, DALIHOM_PVAL, DALIJOINT_PVAL]
pval_cols += [p + '_bh' for p in pval_cols] + [DALIJOINT_RHO]

In [15]:
df = df_all[df_all['Het. AI'] | df_all['Hom. AI'] | df_all['Joint. AI']]
df = df[['chr', 'start', 'end', 'n_cells', 'base_rate', 'qdiff_10'] + pval_cols].reset_index(drop=True)

In [16]:
df['DALIJOINT_RHO'] = 1 - df['DALIJOINT_RHO']

In [17]:
df.to_excel('supplementary_table1.xlsx')

In [18]:
df

,chr,start,end,n_cells,base_rate,qdiff_10,DALIHET_VAE,DALIHOM,DALIJOINT,DALIHET_VAE_bh,DALIHOM_bh,DALIJOINT_bh,DALIJOINT_RHO
0,chr2L,10056155,10057155,1094,0.549698,NaN,0.925040,0.000358,0.000804,0.973230,0.004853,0.009400,0.000000
1,chr2L,10056441,10057441,1431,0.543993,NaN,0.899386,0.000599,0.000992,0.965216,0.007408,0.011085,0.000000
2,chr2L,10056861,10057861,1198,0.539439,NaN,0.700514,0.004859,0.007687,0.893173,0.036865,0.051689,0.000000
3,chr2L,10209280,10210280,1281,0.555198,NaN,0.189010,0.002309,0.002494,0.639445,0.021133,0.022744,0.333333
4,chr2L,10209883,10210883,1031,0.564842,NaN,0.646358,0.000377,0.000437,0.874372,0.005064,0.005752,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8806,chr3R,9926907,9927907,748,0.456506,NaN,0.036037,0.100719,0.014128,0.411720,0.293201,0.079250,0.777778
8807,chr3R,9985324,9986324,885,0.535828,NaN,0.092381,0.008634,0.022363,0.537567,0.056202,0.108883,0.222222
8808,chr3R,9985565,9986565,885,0.535828,NaN,0.092381,0.008634,0.022363,0.537567,0.056202,0.108883,0.222222
8809,chr3R,9985834,9986834,885,0.535828,NaN,0.092381,0.008634,0.022363,0.537567,0.056202,0.108883,0.222222
